In [1]:
import pandas as pd
import numpy as np

In [2]:
# load impressions for 2022
#df_2023_q1 = pd.read_csv('../datasets/impressions_extended_2023_01_02.csv')
#dfs = [df_2022_q1, df_2022_q2, df_2022_q3, df_2022_q4, df_2023_q1]
#impressions_df = pd.concat(dfs)

impressions_df = pd.read_csv('../datasets/omg_2022_2023_data.csv')
impressions_df.head()

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration
0,c445d3c2-4998-4c22-874a-8d45a6e5e619,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,True,0
1,20360deb-aea1-4b77-8381-e9d1ac274ec3,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0
2,47f34dc0-d820-4b7a-a2fd-5c04366553b5,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0
3,d34024ed-0ec8-43a1-bbdf-4765b7ca3daa,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0
4,fd6b2ef6-f470-4ce7-aaa7-597a8c564068,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0


In [3]:
# adding size column
def get_impression_size(row):
    return f"{row['ad_width_chosen']}x{row['ad_height_chosen']}"

impressions_df['size'] = impressions_df.apply(lambda x: get_impression_size(x), axis=1)
impressions_df.head()

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size
0,c445d3c2-4998-4c22-874a-8d45a6e5e619,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,True,0,180x150
1,20360deb-aea1-4b77-8381-e9d1ac274ec3,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600
2,47f34dc0-d820-4b7a-a2fd-5c04366553b5,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600
3,d34024ed-0ec8-43a1-bbdf-4765b7ca3daa,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600
4,fd6b2ef6-f470-4ce7-aaa7-597a8c564068,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600


In [4]:
impressions_df['placement_ids'].value_counts()

{"tag_id":[],"bam_ad_slot":["","",""]}                                                                       16272234
{"tag_id":["19499417"],"bam_ad_slot":["","",""]}                                                               165907
{"tag_id":[],"bam_ad_slot":["insider","",""]}                                                                  156783
{"tag_id":["19499443"],"bam_ad_slot":["","",""]}                                                               146362
{"tag_id":[],"bam_ad_slot":["module","",""]}                                                                   144906
                                                                                                               ...   
{"tag_id":["nyheter24_mobile-panorama-1_18ceaf2c-f348-4587-99f5-327c2513a1f3"],"bam_ad_slot":["","",""]}            1
{"tag_id":["nyheter24_mobile-panorama-1_a9e32d3e-6bf0-4ad2-add2-143b69d1f002"],"bam_ad_slot":["","",""]}            1
{"tag_id":["nyheter24_mobile-panorama-1_60a7f017-f53b-47

In [5]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def len_pids(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    return len(placement_id["tag_id"]) + len(final_bam_ad_slots)

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_len'] = impressions_df['placement_ids'].apply(len_pids)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [6]:
# filter only impressions with int placement IDs
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
# convert placement ids that can be converted to the int to be int, not string
# type(impressions_df.iloc[0]['pid'])     # check this, right now they're strings
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))
impressions_df.head()

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size,pid,pid_len,pid_type
1,20360deb-aea1-4b77-8381-e9d1ac274ec3,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499443,1,int
2,47f34dc0-d820-4b7a-a2fd-5c04366553b5,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499442,1,int
3,d34024ed-0ec8-43a1-bbdf-4765b7ca3daa,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499442,1,int
4,fd6b2ef6-f470-4ce7-aaa7-597a8c564068,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499443,1,int
200,9a949029-6186-4961-9258-62a35f81d89f,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",1180,240,False,False,0,1180x240,19499441,1,int


In [7]:
temp_data = impressions_df[impressions_df['pid'] == 12005770]
temp_data['ad_width_chosen'].value_counts()

180    2
300    1
Name: ad_width_chosen, dtype: int64

In [8]:
impressions_df['pid'].value_counts()

19499417    165907
19499443    146362
19499442    127383
19499441    123226
19499428    111599
             ...  
7222300          1
7899741          1
22938068         1
18647393         1
23808073         1
Name: pid, Length: 11544, dtype: int64

In [9]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    
    for i in range(1, 13):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2022/{i}'):
            for h in os.listdir(f'../predictions/2022/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2022/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [10]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)
impressions_df

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size,pid,pid_len,pid_type,final_fixation
1,20360deb-aea1-4b77-8381-e9d1ac274ec3,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499443,1,int,True
2,47f34dc0-d820-4b7a-a2fd-5c04366553b5,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499442,1,int,True
3,d34024ed-0ec8-43a1-bbdf-4765b7ca3daa,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499442,1,int,False
4,fd6b2ef6-f470-4ce7-aaa7-597a8c564068,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",160,600,False,False,0,160x600,19499443,1,int,True
200,9a949029-6186-4961-9258-62a35f81d89f,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",1180,240,False,False,0,1180x240,19499441,1,int,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21332419,d938f59f-63ac-4ff6-bc9d-881dae1ea5ba,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",160,600,False,True,0,160x600,19499442,1,int,False
21332423,51b5222e-16ee-4239-a324-468cb2afe6c2,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",980,240,False,False,0,980x240,19499441,1,int,False
21332428,5f3ed0fa-5d0a-47e9-b3d0-d530e10165dc,aftonbladet.se,"{""tag_id"":[""19499417""],""bam_ad_slot"":["""","""",""""]}",160,600,False,True,0,160x600,19499417,1,int,False
21332430,0faa07d9-8718-45ae-bf5d-20a27754d718,aftonbladet.se,"{""tag_id"":[""19499428""],""bam_ad_slot"":["""","""",""""]}",468,240,False,True,0,468x240,19499428,1,int,False


In [11]:
# load omd dataset
# df_client_omd = pd.read_excel('../datasets/report - seller-publisher.xlsx')
# df_client_omd

In [12]:
# filter only impressions with tele2 placement IDs
# omd_placements = list(df_client_omd['Placement Id'].unique())
# omd_impressions = impressions_df[impressions_df['pid'].isin(omd_placements)]
# omd_impressions

In [13]:
impressions_df['total_fixation_duration'].value_counts()

0        2408719
11          2118
30          1898
22          1867
23          1229
          ...   
97224          1
5604           1
97635          1
6219           1
25007          1
Name: total_fixation_duration, Length: 11453, dtype: int64

In [14]:
# groupby and aggregate
grouped_df = impressions_df.groupby(['hostname', 'pid', 'size']).apply(
    lambda x: pd.Series(
        {
            'num_impressions' : x['id'].count(),
            'num_fixations': x.loc[x['final_fixation'], 'id'].count(),
            'average_fixation_duration': x['total_fixation_duration'].mean(),
            'average_fixation_duration_filtered': x[x['total_fixation_duration'] != 0]['total_fixation_duration'].mean(),
#             'width': x['ad_width_chosen'].mean(),
#             'height': x['ad_height_chosen'].mean()
        }
    )
)
grouped_df
# grouped_df[grouped_df.loc['average_fixation_duration'] != 0 & grouped_df.loc['num_impressions'] == 0]

num_impressions  \
hostname                      pid      size                        
1000logos.net                 19680007 728x90                2.0   
1001fonts.com                 12005727 1180x240              3.0   
                                       180x150               1.0   
                              12005770 180x150               2.0   
                                       300x250               1.0   
...                                                          ...   
zt2downloadlibrary.fandom.com 27472329 180x150               4.0   
                                       300x250               1.0   
                              27488081 180x150               1.0   
                                       250x480               2.0   
                              27488083 180x150               1.0   

                                                 num_fixations  \
hostname                      pid      size                      
1000logos.net                 19680007 728x90              0.0   
1001fonts.com                 12005727 1180x240            0.0   
                                       180x150             0.0   
                              12005770 180x150             0.0   
                                       300x250             0.0   
...                                                        ...   
zt2downloadlibrary.fandom.com 27472329 180x150             0.0   
                                       300x250             0.0   
                              27488081 180x150             0.0   
                                       250x480             0.0   
                              27488083 180x150             0.0   

                                                 average_fixation_duration  \
hostname                      pid      size                                  
1000logos.net                 19680007 728x90                          0.0   
1001fonts.com                 12005727 1180x240                        0.0   
                                       180x150                         0.0   
                              12005770 180x150                         0.0   
                                       300x250                         0.0   
...                                                                    ...   
zt2downloadlibrary.fandom.com 27472329 180x150                         0.0   
                                       300x250                         0.0   
                              27488081 180x150                         0.0   
                                       250x480                         0.0   
                              27488083 180x150                         0.0   

                                                 average_fixation_duration_filtered  
hostname                      pid      size                                          
1000logos.net                 19680007 728x90                                   NaN  
1001fonts.com                 12005727 1180x240                                 NaN  
                                       180x150                                  NaN  
                              12005770 180x150                                  NaN  
                                       300x250                                  NaN  
...                                                                             ...  
zt2downloadlibrary.fandom.com 27472329 180x150                                  NaN  
                                       300x250                                  NaN  
                              27488081 180x150                                  NaN  
                                       250x480                                  NaN  
                              27488083 180x150                                  NaN  

[33039 rows x 4 columns]

In [15]:
grouped_df['ratio'] = round(grouped_df['num_fixations'] / grouped_df['num_impressions'], 4)
grouped_df['ratio_percent'] = round(grouped_df['ratio'] * 100, 2)
grouped_df['average_fixation_duration'] = round(grouped_df['average_fixation_duration'], 2)
grouped_df['average_fixation_duration_filtered'] = round(grouped_df['average_fixation_duration_filtered'], 2)
grouped_df

num_impressions  \
hostname                      pid      size                        
1000logos.net                 19680007 728x90                2.0   
1001fonts.com                 12005727 1180x240              3.0   
                                       180x150               1.0   
                              12005770 180x150               2.0   
                                       300x250               1.0   
...                                                          ...   
zt2downloadlibrary.fandom.com 27472329 180x150               4.0   
                                       300x250               1.0   
                              27488081 180x150               1.0   
                                       250x480               2.0   
                              27488083 180x150               1.0   

                                                 num_fixations  \
hostname                      pid      size                      
1000logos.net                 19680007 728x90              0.0   
1001fonts.com                 12005727 1180x240            0.0   
                                       180x150             0.0   
                              12005770 180x150             0.0   
                                       300x250             0.0   
...                                                        ...   
zt2downloadlibrary.fandom.com 27472329 180x150             0.0   
                                       300x250             0.0   
                              27488081 180x150             0.0   
                                       250x480             0.0   
                              27488083 180x150             0.0   

                                                 average_fixation_duration  \
hostname                      pid      size                                  
1000logos.net                 19680007 728x90                          0.0   
1001fonts.com                 12005727 1180x240                        0.0   
                                       180x150                         0.0   
                              12005770 180x150                         0.0   
                                       300x250                         0.0   
...                                                                    ...   
zt2downloadlibrary.fandom.com 27472329 180x150                         0.0   
                                       300x250                         0.0   
                              27488081 180x150                         0.0   
                                       250x480                         0.0   
                              27488083 180x150                         0.0   

                                                 average_fixation_duration_filtered  \
hostname                      pid      size                                           
1000logos.net                 19680007 728x90                                   NaN   
1001fonts.com                 12005727 1180x240                                 NaN   
                                       180x150                                  NaN   
                              12005770 180x150                                  NaN   
                                       300x250                                  NaN   
...                                                                             ...   
zt2downloadlibrary.fandom.com 27472329 180x150                                  NaN   
                                       300x250                                  NaN   
                              27488081 180x150                                  NaN   
                                       250x480                                  NaN   
                              27488083 180x150                                  NaN   

                                                 ratio  ratio_percent  
hostname                      pid      size                            
1000logos.net 

In [16]:
result_df = grouped_df.reset_index()
result_df.rename(columns={'pid': 'placement_id'}, inplace=True)
result_df

,hostname,placement_id,size,num_impressions,num_fixations,average_fixation_duration,average_fixation_duration_filtered,ratio,ratio_percent
0,1000logos.net,19680007,728x90,2.0,0.0,0.0,NaN,0.0,0.0
1,1001fonts.com,12005727,1180x240,3.0,0.0,0.0,NaN,0.0,0.0
2,1001fonts.com,12005727,180x150,1.0,0.0,0.0,NaN,0.0,0.0
3,1001fonts.com,12005770,180x150,2.0,0.0,0.0,NaN,0.0,0.0
4,1001fonts.com,12005770,300x250,1.0,0.0,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
33034,zt2downloadlibrary.fandom.com,27472329,180x150,4.0,0.0,0.0,NaN,0.0,0.0
33035,zt2downloadlibrary.fandom.com,27472329,300x250,1.0,0.0,0.0,NaN,0.0,0.0
33036,zt2downloadlibrary.fandom.com,27488081,180x150,1.0,0.0,0.0,NaN,0.0,0.0
33037,zt2downloadlibrary.fandom.com,27488081,250x480,2.0,0.0,0.0,NaN,0.0,0.0


In [17]:
# load omd dataset
df_client_omd = pd.read_excel('../datasets/report - seller-publisher.xlsx')
df_client_omd

,Seller,Seller Name,Publisher Id,Publisher Name,Placement Id,Placement Name,Member Currency,Imps
0,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499475,SE-Aftonbladet-wph-Front-Panorama_1,SEK,249128
1,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499490,SE-Aftonbladet-wph-Front-Toppanorama,SEK,184180
2,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499428,SE-Aftonbladet-wde-Front-Modul_1,SEK,217728
3,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499363,SE-Aftonbladet-apa-Front-Toppanorama,SEK,92770
4,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499470,SE-Aftonbladet-wph-Article-Toppanorama,SEK,131159
...,...,...,...,...,...,...,...,...
281,9943,Schibsted Sweden,1431700,SE-Bytbil,14746565,SE-BytBil-wph-Sok_Lista_Fordon_Panorama_2,SEK,1481
282,12099,BONNIER NEWS AB,1856396,MM Bandypuls,20148910,Mob,SEK,1122
283,9943,Schibsted Sweden,1403613,SE-TV,19499789,SE-TV-wde-Other-Toppanorama,SEK,643
284,12099,BONNIER NEWS AB,1856377,BNS Expressen,20615026,Mob_top,SEK,878


In [30]:
# merge impressions with clients data
result_df = result_df.merge(df_client_omd,  left_on='placement_id', right_on='Placement Id', how='inner')
result_df

,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,average_fixation_duration_filtered,eyetracker_on,Seller,Seller Name,Publisher Id,Publisher Name,Placement Id,Placement Name,Member Currency,Imps
0,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,180x150,0.00,0.00,0.00,True,12099,BONNIER NEWS AB,1856392,BMB Allt om mat,20148892,Panorama,SEK,1537
1,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,980x240,0.00,0.00,0.00,True,12099,BONNIER NEWS AB,1856392,BMB Allt om mat,20148892,Panorama,SEK,1537
2,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,180x150,0.00,0.00,0.00,True,12099,BONNIER NEWS AB,1856392,BMB Allt om mat,20148892,Panorama,SEK,1537
3,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,36.36,44.73,246.00,True,12099,BONNIER NEWS AB,1856392,BMB Allt om mat,20148892,Panorama,SEK,1537
4,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x360,42.86,164.24,1149.67,True,12099,BONNIER NEWS AB,1856392,BMB Allt om mat,20148892,Panorama,SEK,1537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,Schibsted Sweden,SE-TV,tv.nu,19499791,160x600,5.03,24.89,494.62,True,9943,Schibsted Sweden,1403613,SE-TV,19499791,SE-TV-wde-Other-Widescreen_2,SEK,871
921,Schibsted Sweden,SE-TV,tv.nu,19499791,180x150,0.00,0.00,0.00,True,9943,Schibsted Sweden,1403613,SE-TV,19499791,SE-TV-wde-Other-Widescreen_2,SEK,871
922,Schibsted Sweden,SE-TV,tv.nu,19499791,250x360,0.00,0.00,0.00,True,9943,Schibsted Sweden,1403613,SE-TV,19499791,SE-TV-wde-Other-Widescreen_2,SEK,871
923,Schibsted Sweden,SE-TV,tv.nu,19499791,500x555,0.00,0.00,0.00,True,9943,Schibsted Sweden,1403613,SE-TV,19499791,SE-TV-wde-Other-Widescreen_2,SEK,871


In [32]:
result_df['Placement Name'].value_counts()

Panorama                                       54
Outsider                                       46
Insider                                        23
Module                                         23
Mob                                            19
                                               ..
SE-Aftonbladet-wde-Article-Modul_Backfill_1     2
SE-Aftonbladet-wph-Bloggar_Interstich           1
SE-Aftonbladet-wph-Bloggar_Panorama_3           1
SE-Aftonbladet-wph-Front-Panorama_18            1
SE-Aftonbladet-wph-Front-Panorama_14            1
Name: Placement Name, Length: 145, dtype: int64

In [33]:
result_df = result_df[[
    'Seller Name', 
    'Publisher Name', 
    'hostname', 
    'placement_id',
    'size',
#     'num_impressions', 
#     'num_fixations', 
    'ratio', 
#    'ratio_percent', 
    'average_fixation_duration',
    'average_fixation_duration_filtered',
]]
result_df.rename(columns={'Seller Name': 'seller_name', 'Publisher Name': 'publisher_name'}, inplace=True)
result_df

/tmp/ipykernel_16401/3982675310.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={'Seller Name': 'seller_name', 'Publisher Name': 'publisher_name'}, inplace=True)


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,average_fixation_duration_filtered
0,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,180x150,0.00,0.00,0.00
1,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,980x240,0.00,0.00,0.00
2,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,180x150,0.00,0.00,0.00
3,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,36.36,44.73,246.00
4,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x360,42.86,164.24,1149.67
...,...,...,...,...,...,...,...,...
920,Schibsted Sweden,SE-TV,tv.nu,19499791,160x600,5.03,24.89,494.62
921,Schibsted Sweden,SE-TV,tv.nu,19499791,180x150,0.00,0.00,0.00
922,Schibsted Sweden,SE-TV,tv.nu,19499791,250x360,0.00,0.00,0.00
923,Schibsted Sweden,SE-TV,tv.nu,19499791,500x555,0.00,0.00,0.00


In [35]:
# sort on seller_name and publisher_name
result_df.sort_values(by=['publisher_name', 'seller_name'], inplace=True)
result_df.rename({'ratio_percent': 'ratio'}, inplace=True)
result_df

/tmp/ipykernel_16401/681984665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.sort_values(by=['publisher_name', 'seller_name'], inplace=True)
/tmp/ipykernel_16401/681984665.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename({'ratio_percent': 'ratio'}, inplace=True)


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,average_fixation_duration_filtered
0,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,180x150,0.00,0.00,0.00
1,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,980x240,0.00,0.00,0.00
2,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,180x150,0.00,0.00,0.00
3,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,36.36,44.73,246.00
4,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x360,42.86,164.24,1149.67
...,...,...,...,...,...,...,...,...
920,Schibsted Sweden,SE-TV,tv.nu,19499791,160x600,5.03,24.89,494.62
921,Schibsted Sweden,SE-TV,tv.nu,19499791,180x150,0.00,0.00,0.00
922,Schibsted Sweden,SE-TV,tv.nu,19499791,250x360,0.00,0.00,0.00
923,Schibsted Sweden,SE-TV,tv.nu,19499791,500x555,0.00,0.00,0.00


In [36]:
# highlight rows with invalid data combinations
# invalid combinations:
#    * ratio == 0 and average_fixation_duration > 0
#    * ratio > 0 and average_fixation_duration == 0

def highlight_rows(row):
    if ((row['ratio'] == 0 and row['average_fixation_duration'] > 0) or (row['ratio'] > 0 and row['average_fixation_duration'] == 0)):
        return False
        # return['background-color: orange']*5
    else:
        return True
        # return['background-color: white']*5

# display(result_df.style.apply(highlight_rows, axis=1))
result_df['eyetracker_on'] = result_df.apply(lambda row: highlight_rows(row), axis=1)
result_df

/tmp/ipykernel_16401/212537132.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['eyetracker_on'] = result_df.apply(lambda row: highlight_rows(row), axis=1)


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,average_fixation_duration_filtered,eyetracker_on
0,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,180x150,0.00,0.00,0.00,True
1,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,980x240,0.00,0.00,0.00,True
2,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,180x150,0.00,0.00,0.00,True
3,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,36.36,44.73,246.00,True
4,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x360,42.86,164.24,1149.67,True
...,...,...,...,...,...,...,...,...,...
920,Schibsted Sweden,SE-TV,tv.nu,19499791,160x600,5.03,24.89,494.62,True
921,Schibsted Sweden,SE-TV,tv.nu,19499791,180x150,0.00,0.00,0.00,True
922,Schibsted Sweden,SE-TV,tv.nu,19499791,250x360,0.00,0.00,0.00,True
923,Schibsted Sweden,SE-TV,tv.nu,19499791,500x555,0.00,0.00,0.00,True


In [24]:
result_df['eyetracker_on'].value_counts()
result_df['ratio'] *= 100

/tmp/ipykernel_16401/3551984963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['ratio'] *= 100


In [37]:
result_df["average_fixation_duration"].value_counts()
result_df["average_fixation_duration_filtered"] = result_df["average_fixation_duration_filtered"].fillna(0)


/tmp/ipykernel_16401/1292919701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["average_fixation_duration_filtered"] = result_df["average_fixation_duration_filtered"].fillna(0)


In [38]:
result_df["publisher_name"].value_counts()

SE-Aftonbladet         381
SE-SvD                 137
BNS Expressen           80
SE-TV                   51
SE-Klart                49
BNS Dagens nyheter      42
SE-Prisjakt             41
SE-Blocket              40
SE-Omni                 32
SE-Bytbil               18
BNS Sydsvenskan         11
BNS HD                   8
SE-Godare                6
BMB Allt om mat          5
MM NA                    5
BMB Amelia               4
BMB Mama                 4
MM VLT                   3
BMB Tailsweep            3
BNS Dagens industri      2
BMB Damernas värld       2
BMB Teknikens värld      1
Name: publisher_name, dtype: int64

In [39]:
# OMG and GroupM are competitors, but they use the same dataformat
result_df.to_excel('../final_reports/groupM_performance_report.xlsx', index=False)
result_df.to_csv('../final_reports/gourpM_performance_report.csv', index=False)

In [55]:
len(result_df['ratio'].value_counts())

801

In [60]:
result_df

,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,average_fixation_duration_filtered,eyetracker_on
1376,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,180x150,0.00,0.00,NaN,True
1377,BONNIER NEWS AB,BMB Allt om mat,alltomtradgard.expressen.se,20148892,980x240,0.00,0.00,NaN,True
9419,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,180x150,0.00,0.00,NaN,True
9420,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,36.36,44.73,246.00,True
9421,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x360,42.86,164.24,1149.67,True
...,...,...,...,...,...,...,...,...,...
33034,NaN,NaN,zt2downloadlibrary.fandom.com,27472329,180x150,0.00,0.00,NaN,True
33035,NaN,NaN,zt2downloadlibrary.fandom.com,27472329,300x250,0.00,0.00,NaN,True
33036,NaN,NaN,zt2downloadlibrary.fandom.com,27488081,180x150,0.00,0.00,NaN,True
33037,NaN,NaN,zt2downloadlibrary.fandom.com,27488081,250x480,0.00,0.00,NaN,True
